In [1]:
import pandas as pd
import yfinance as yf
import numpy as np

from datetime import datetime
from dateutil.relativedelta import relativedelta
from time import strptime

pd.set_option('display.max_rows', None)  # 모든 행 출
pd.set_option('display.max_columns', None)  # 모든 열 출력
pd.set_option('display.expand_frame_repr', None)  # 긴 데이터 프레임 줄바꿈 없이 출력

symbols = ['SPY', 'IWM']
start_date = '2015-12-01'
end_date = '2016-12-31'

In [2]:
# 배당 데이터 가져오기
def get_dividends(symbols:list):
    result_dfs = {}

    for symbol in symbols:
        info = yf.Ticker(symbol)
        df = info.actions.reset_index()
        df['Date'] = df['Date'].dt.date # 년월일만 남김
        df['Symbol'] = symbol
        df = df[['Date','Symbol','Dividends']]
        result_dfs[symbol]=df

    return result_dfs

In [3]:
# 주가 데이터 가져오기
def get_stock_data(symbols:list, start_date, end_date):
    
    result_dfs = {}
    
    # 로딩하는건 progress = False 하면 해제됨
    df = yf.download(tickers=symbols, start=start_date, end=end_date, progress=False, auto_adjust=False)

    # 배당 데이터 가져오기
    dividends_dfs = get_dividends(symbols= symbols)

    for symbol in symbols:
        # 특정 심볼 선택
        df_symbol = df.loc[:, df.columns.get_level_values(1) == symbol]

        # 멀티 column의 심볼들 제거 후, 심볼 추가
        df_symbol.columns = df_symbol.columns.droplevel(1)
        df_symbol = df_symbol.reset_index()
        df_symbol .columns.name = None

        # 심볼 추가 및 column 위치 조정
        df_symbol['Symbol'] = symbol
        columns = [col for col in df_symbol.columns if col != 'Date' and col != 'Symbol']
        columns = ['Date', 'Symbol'] + columns 
        df_symbol = df_symbol[columns]

        # Date값을 datetime 으로 변경
        df_symbol['Date'] = pd.to_datetime(df_symbol['Date'])

        df_dividends = dividends_dfs[symbol]
        df_dividends['Date'] = pd.to_datetime(df_dividends['Date'])

        # 배당데이터 병합
        df_symbol = df_symbol.merge(df_dividends, on=['Date','Symbol'], how = 'left')
        df_symbol.fillna(0, inplace=True)
        result_dfs[symbol] = df_symbol

    return result_dfs

In [4]:
# 종가 데이터 가져오기
def get_stock_data_close(symbols:list, start_date, end_date=None, include_diviend=False, key = 'Close'):
    
    result_dfs = {}

    symbol_dfs = get_stock_data(symbols=symbols, start_date=start_date,end_date=end_date)
    
    for symbol, df_symbol in symbol_dfs.items():

        if key == 'Adj Close':
           result_close = df_symbol['Adj Close']
           condition = 'Adj Close'
        
        elif key == 'Close':
            if include_diviend == True:
                result_close = df_symbol['Close'] + df_symbol['Dividends']
                condition = 'Close + Dividends'
            else:
                result_close = df_symbol['Close']
                condition = 'Close'

        df_symbol['Result Close'] = result_close
        df_symbol['Condition'] = condition
        df_symbol = df_symbol[['Date','Symbol','Adj Close', 'Close', 'Dividends','Result Close','Condition']]

        result_dfs[symbol] = df_symbol

    return result_dfs


# get_stock_data_close(symbols=symbols, start_date=start_date, include_diviend=True)

In [5]:
def get_stock_data_with_ma(symbols:list, start_date, end_date, mas, type='ma_day'):

    result_dfs = {}

    if len(mas) <= 0:
        print('mas count is 0!')
        return
    
    if start_date != None:
       start_date = datetime.strptime(start_date, "%Y-%m-%d") if isinstance(start_date, str) else start_date
    
    if end_date != None:
       end_date = datetime.strptime(end_date, "%Y-%m-%d") if isinstance(end_date, str) else end_date

    sort_mas = sorted(mas)

    # 일별이동평균
    if type == 'ma_day':
        prev_day = sort_mas[-1]
        prev_date = start_date - relativedelta(days = prev_day*1.5)

    
    # 월별이동평균 - 해당월의 마지막날 데이터의 종가 기준으로 평균 값 구함.
    elif type == 'ma_month':
        prev_day = sort_mas[-1]*25
        prev_date = start_date -relativedelta(days=prev_day*1.5)

    # 월말 데이터 가져오기
    symbol_dfs = get_stock_data_close(symbols=symbols, start_date=prev_date, end_date=end_date, include_diviend=True, key= 'Close')

    for symbol, df_symbol in symbol_dfs.items():
        df = df_symbol.copy()

        if type == 'ma_day':
            for ma in mas:
                df[f'SMA_{ma}'] = df['Result Close'].rolling(window=ma).mean()

        elif type == 'ma_month':
            monthly_df = df.groupby(df['Date'].dt.to_period('M')).apply(lambda x: x.iloc[-1]).reset_index(drop=True)


            new_columes = []
            for ma in mas:
                col = f'MMA_{ma}'
                monthly_df[col] = monthly_df['Result Close'].rolling(window=ma).mean()
                new_columes.append(col)
                
            
            df = df.merge(monthly_df[['Date', 'Symbol'] + new_columes], on=['Date', 'Symbol'], how='left')
            
            
        # 기간범위 체크
        df = df[df['Date'] >= start_date]
        if end_date is not None:
            df = df[df['Date'] <= end_date]

        result_dfs[symbol] = df

    return result_dfs
            
# get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=None,mas=[10], type='ma_day')

In [6]:
# 월말 데이터 가져오기
def filter_close_last_month(symbol_dfs):
    result_dfs = {}

    for symbol, df_symbol in symbol_dfs.items():
        monthly_df = df_symbol.groupby(df_symbol['Date'].dt.to_period('M')).apply(lambda x: x.iloc[-1]).reset_index(drop=True)
        
        result_dfs[symbol] = monthly_df

    return result_dfs

In [7]:
# 수익률 구하기
def get_profit_ratio(symbol_dfs: dict, init_balance: float = 10000) -> dict:
    result_dfs = {}

    for symbol, df_symbol in symbol_dfs.items():
        df_symbol = df_symbol.copy()

        # 컬럼 초기화
        df_symbol['Profit Ratio'] = 0.0
        df_symbol['Balance'] = 0.0

        # 초기값 설정
        df_symbol.loc[0, 'Balance'] = init_balance
        df_symbol.loc[0, 'Profit Ratio'] = 0.0

        for i in range(1, len(df_symbol)):
            prev_close = df_symbol.loc[i - 1, 'Result Close']
            curr_close = df_symbol.loc[i, 'Result Close']

            if pd.notna(prev_close) and prev_close != 0:
                profit_ratio = (curr_close - prev_close) / prev_close
            else:
                profit_ratio = 0.0

            df_symbol.loc[i, 'Profit Ratio'] = round(profit_ratio * 100, 2)
            df_symbol.loc[i, 'Balance'] = round(df_symbol.loc[i - 1, 'Balance'] * (1 + profit_ratio), 2)

        result_dfs[symbol] = df_symbol
        display(df_symbol)

    return result_dfs

# df = get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=end_date,mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# get_profit_ratio(df)

In [8]:
def get_profit_ratio_avg(symbol_dfs:dict, intervals=[1,3,6]):

    result_dfs = {}

    for symbol, df in symbol_dfs.items():
        
        interval_dfs ={}
        for interval in intervals:
            copy_df = df.copy()
            shift_df = copy_df.shift(interval)
            
            copy_df['Interval'] = interval

            profit_value = copy_df['Result Close'] - shift_df['Result Close']

            copy_df['Profit Value'] = profit_value
            copy_df['Profit Ratio'] = ((profit_value/shift_df['Result Close'])*100).round(2)

            interval_dfs[interval] = copy_df
            
        
        avg_profit_ratios = sum(df_interval['Profit Ratio'] for df_interval in interval_dfs.values()) / len(intervals)
        
        new_df = df.copy()
        new_df['Avg Profit Ratio'] = avg_profit_ratios
        new_df['Interval'] = [intervals] * len(df)

        result_dfs[symbol] = new_df

    return result_dfs

# df = get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=end_date,mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# get_profit_ratio_avg(df)

In [22]:
def merge_to_dfs(symbol_dfs:dict):
    all_dfs = []
    for symbol, df_symbol in symbol_dfs.items():
        df = df_symbol.copy()
        all_dfs.append(df)

    merged_df = pd.concat(all_dfs, ignore_index=True)
    group_df = merged_df.groupby('Date').agg(lambda x: list(x)).reset_index()

    for col in group_df.columns:
        group_df[col] = group_df[col].apply(
            lambda x: [round(val, 2) if isinstance(val, (int, float)) else val for val in x] if isinstance(x, list) else x
        )

    return group_df

In [10]:
# 리벨런싱(균등) - 연간 N회 리벨런싱. 균등하게
def rebalancing_evenly(symbol_dfs: dict, ratios:list, interval:int = 12, init_balance = 10000):

    if len(symbol_dfs) != len(ratios):
        print('심볼이랑, 가중치랑 길이가 다름!')
        return
    
    if sum(ratios) != 100:
        print(f'가중치 값이 다름! : {sum(ratios)}')
        return

    df = merge_to_dfs(symbol_dfs)
    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition']]
    df = df[columns]

    total_ratio = sum(ratios)
    start_balances = [round((r / total_ratio) * init_balance, 2) for r in ratios]
    end_balances = [0]*len(ratios)

    # 초기값 설정
    df['End Balance'] = None
    df['Restart Balance'] = None
    df['Balance'] = None
    df['Can Rebalance'] = False

    df.at[0, 'End Balance'] = end_balances
    df.at[0, 'Restart Balance'] = start_balances
    df.at[0, 'Balance'] = init_balance

    for i in range(1, len(df)):
        # interval 값만큼 건너뛰고 리벨런싱
        # 각 종목의 증감률을 Result Close에서 계산
        result_close = df.at[i, 'Result Close']
        change_factors = [result_close[j]/ df.at[i-1,'Result Close'][j] for j in range(len(result_close))]

        end_balances = [round(df.at[i-1, 'Restart Balance'][j] * change_factors[j], 2) for j in range(len(change_factors))]
        df.at[i, 'End Balance'] = end_balances

        next_balance = sum(end_balances)
        df.at[i, 'Balance'] = int(next_balance)
        
        if i % interval == 0:
            restart_balance = [round((r/100)*next_balance,2) for r in ratios]
            df.at[i, 'Restart Balance'] = restart_balance
            df.at[i, 'Can Rebalance'] = True
        else:
            df.at[i, 'Restart Balance'] = end_balances
            

    return df
        

# df = get_stock_data_with_ma(symbols=symbols,start_date=start_date,end_date=end_date,mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# rebalancing_evenly(df, ratios=[50,50], interval=2)

In [11]:
def get_performance(df):
    # 심볼과 기본 데이터 설정
    symbol = df['Symbol'].iloc[0]
    start_balance = df['Balance'].dropna().iloc[0]
    end_balance = df['Balance'].dropna().iloc[-1]
    start_date = pd.to_datetime(df['Date'].iloc[0])
    end_date = pd.to_datetime(df['Date'].iloc[-1])
    num_months = (end_date.year - start_date.year) * 12 + (end_date.month - start_date.month)

    # 연복리 성장률 (CAGR) 계산
    years = num_months / 12
    CAGR = ((end_balance / start_balance) ** (1 / years)) - 1

    # 월간 수익률 계산
    balance_series = df['Balance'].dropna().astype(float)
    monthly_returns = balance_series.pct_change().dropna()

    # 무위험 수익률 가정 (연 2%를 월간으로 환산)
    risk_free_rate = 0.02 / 12

    # 연간화된 표준 편차 계산
    annualized_std_dev = monthly_returns.std() * np.sqrt(12)

    # Sharpe Ratio 계산
    sharpe_ratio = (monthly_returns.mean() - risk_free_rate) / monthly_returns.std() * np.sqrt(12)

    # 최대 낙폭 (MDD) 계산
    running_max = np.maximum.accumulate(balance_series)
    drawdown = (balance_series / running_max - 1).min()

    # 결과 데이터프레임 생성
    results = pd.DataFrame({
        "Symbol": [symbol],
        "Start Date" : [start_date],
        "End Date":[end_date],
        "Start Balance":[start_balance],
        "End Balance": [end_balance],
        "Annualized Return (CAGR)": [CAGR],
        "Standard Deviation": [annualized_std_dev],
        "Sharpe Ratio": [sharpe_ratio],
        "Maximum Drawdown": [drawdown]
    })

    return results


# df = get_stock_data_with_ma(symbols=['VTI','TLT','BIL','GLD'], start_date='2015-12-01', end_date='2024-12-31', mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# df = rebalancing_evenly(df, ratios=[25,25,25,25], interval=12)
# get_performance(df)
    

In [12]:
# 해리브라운의 영구포트폴리오
def harry_browne_permanent_portfolio():
    df = get_stock_data_with_ma(symbols=['VTI','TLT','BIL','GLD'], start_date='2007-12-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = rebalancing_evenly(df, ratios=[25,25,25,25], interval=12)
    df = get_performance(df)
    display(df)

# 레이달리오 올시즌 포트폴리오
def ray_dalio_all_seasons():
    df = get_stock_data_with_ma(symbols=['VTI','TLT','IEF','GSG','GLD'], start_date='2015-12-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = rebalancing_evenly(df, ratios=[30,40,15,7.5, 7.5], interval=12)
    df = get_performance(df)
    display(df)


harry_browne_permanent_portfolio()
ray_dalio_all_seasons()


,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[VTI, TLT, BIL, GLD]",2007-12-31,2025-03-28,10000,21805,0.046228,0.07495,0.374507,-0.164706


,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[VTI, TLT, IEF, GSG, GLD]",2015-12-31,2025-03-28,10000,13856,0.035887,0.090016,0.21483,-0.234562


In [13]:
# 선택한 자산군 중 기간월 평균 수익률이 높은 N개를 추출
# N개를 균등하게 배분
# N개의 각 자산들의 10개월 이동평균보다 낮다면 현금으로 보유
# 월 1회 리벨런싱
def rebalancing_taa(symbol_dfs: dict, rank = 3, init_balance = 10000):
    
    # 'Restart Asset' 생성 함수 (조건 추가)
    def get_top_assets(row):
        profit_ratios = row['Avg Profit Ratio']
        symbols = row['Symbol']
        result_close = row['Result Close']
        mma_10 = row['MMA_10']
        
        # 높은 순으로 인덱스 정렬 후 상위 rank개 추출
        top_indices = sorted(range(len(profit_ratios)), key=lambda i: profit_ratios[i], reverse=True)[:rank]
        
        # 조건에 맞는 symbol만 추가
        selected_assets = [
            symbols[i] for i in top_indices
            if result_close[i] >= mma_10[i]
        ]
        
        return selected_assets


    for symbol, df in symbol_dfs.items():
        symbol_dfs[symbol] = df.dropna(subset=['Avg Profit Ratio'])

    df = merge_to_dfs(symbol_dfs)
    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval']]
    df = df[columns]

    df['End Balance'] = None
    df['End Cash'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_top_assets, axis=1) 
    df['Restart Balance'] = None
    
    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']
        
        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            end_cash = df.at[idx-1, 'Cash']
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance]) + end_cash

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'End Cash'] = end_cash
            df.at[idx, 'Balance'] = balance
            

        split_balance = int(balance / rank)
        restart_balance = []
        if len(restart_asset) > 0:
            for _ in  range(len(restart_asset)):
                restart_balance.append(split_balance)
                balance -= split_balance

        df.at[idx, 'Restart Balance'] = restart_balance
        df.at[idx, 'Cash'] = balance
                 
    return df


# symbols = ['SPY','IWD','IWM', 'IWN','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','BWX','GLD']
# symbols = ['SPY','IWM','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','GLD']
# df = get_stock_data_with_ma(symbols=symbols, start_date='2014-11-01', end_date='2024-12-31', mas=[10], type='ma_month')
# df = filter_close_last_month(df)
# df = get_profit_ratio_avg(df, [1,3,6,12])
# df = rebalancing_taa(df)
# display(df)
# display(get_performance(df))


In [15]:
# 공격형 TAA전략
def gtaa():
    # symbols = ['SPY','IWD','IWM', 'IWN','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','BWX','GLD']
    symbols = ['SPY','IWM','MTUM','EFA','TLT','IEF','LQD','DBC','VNQ','GLD']
    start_date = '2014-12-01'

    df = get_stock_data_with_ma(symbols=symbols, start_date=start_date, end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_profit_ratio_avg(df, [1,3,6,12])
    df = rebalancing_taa(df, 3)
    # display(df)
    display(get_performance(df))

In [45]:
# 오리지널 듀얼모멘텀 전략
# SPY, EFA(선진국주식), AGG(미국채권) 투자
# 매월, SPY, EFA, BIL(초단기채권)의 최근 12개월 수익률 계산
# 수익률이 SPY > BIL 일 경우, SPY, EFA중 수익률 높은 ETF에 투자.
# BIL < SPY일 경우, AGG에 투자

def original_dual_momentum():

    def get_restart_asset(row):
        symbols = row['Symbol']
        profit_ratios = row['Avg Profit Ratio']
        
        # 종목 이름과 인덱스를 매핑
        symbol_to_index = {symbol: i for i, symbol in enumerate(symbols)}
        
        # 필요한 종목들이 모두 있는지 확인
        required_symbols = ['SPY', 'BIL', 'EFA', 'AGG']
        if not all(sym in symbol_to_index for sym in required_symbols):
            return []  # 조건에 맞는 종목이 없으면 빈 리스트 반환

        spy_idx = symbol_to_index['SPY']
        bil_idx = symbol_to_index['BIL']
        efa_idx = symbol_to_index['EFA']
        agg_idx = symbol_to_index['AGG']
        
        spy_ratio = profit_ratios[spy_idx]
        bil_ratio = profit_ratios[bil_idx]
        efa_ratio = profit_ratios[efa_idx]
        agg_ratio = profit_ratios[agg_idx]  # 필요하면 사용 가능

        if spy_ratio > bil_ratio:
            selected = 'SPY' if spy_ratio >= efa_ratio else 'EFA'
            return [selected]
        else:
            return ['AGG']


    init_balance = 10000
    symbols = ['SPY','EFA','AGG','BIL']
    df = get_stock_data_with_ma(symbols=symbols, start_date='2007-08-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_profit_ratio_avg(df, [12])


    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Avg Profit Ratio'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval']]
    df = df[columns]

    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_restart_asset, axis=1) 
    df['Restart Balance'] = None

    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']

        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance])

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'Balance'] = balance
        
        split_balance = int(balance / len(restart_asset))
        restart_balance = []
        if len(restart_asset) > 0:
            for _ in  range(len(restart_asset)):
                restart_balance.append(split_balance)
                balance -= split_balance

        df.at[idx, 'Restart Balance'] = restart_balance

    df = get_performance(df)
    display(df)


original_dual_momentum()

,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, EFA, AGG, BIL]",2008-08-29,2025-03-28,10000,25676,0.05851,0.123904,0.360771,-0.203766


In [ ]:
# 종합 듀얼 모멘텀
# 포트폴리오를 4개 파트로 나눔
# 1. 주식 - SPY, ETF(해외주식)
# 2. 채권 - LQD(회사채), HYD(미국 하이일드 채권)
# 3. 부동산 - VNQ(부동산 리츠), REM(모기지 리츠)
# 4. 불경기 - TLT, GLD
# 각 파트별 1개씩 투자함 (25% 배분)
# 최근 12개월 수익률을 비교해서 1개씩 선별함.
# 근데, ETF 수익 모두가 BIL 수익보다 낮으면,
def composite_dual_momentum():

    def get_selected_assets(row):
        symbols = row['Symbol']
        profit_ratios = row['Avg Profit Ratio']
        
        # 종목과 인덱스를 매핑
        symbol_to_index = {symbol: i for i, symbol in enumerate(symbols)}
        
        # BIL 수익률
        bil_ratio = profit_ratios[symbol_to_index['BIL']] if 'BIL' in symbol_to_index else None

        if bil_ratio is None:
            return []  # BIL 없으면 처리 안 함

        # 자산군 정의
        asset_groups = {
            'stock': ['SPY', 'EFA'],
            'bond': ['LQD', 'HYG'],
            'real_estate': ['VNQ', 'REM'],
            'recession': ['TLT', 'GLD']
        }

        selected_assets = []

        for group, candidates in asset_groups.items():
            valid_candidates = [
                (symbol, profit_ratios[symbol_to_index[symbol]])
                for symbol in candidates
                if symbol in symbol_to_index
            ]

            # 유효한 후보 없으면 해당 그룹 스킵
            if not valid_candidates:
                continue

            # 후보 중 수익률이 가장 높은 ETF 찾기
            best_symbol, best_ratio = max(valid_candidates, key=lambda x: x[1])

            # 해당 그룹의 모든 ETF 수익률이 BIL보다 낮은지 확인
            all_lower_than_bil = all(ratio < bil_ratio for _, ratio in valid_candidates)

            if all_lower_than_bil:
                selected_assets.append('BIL')
            else:
                selected_assets.append(best_symbol)

        return selected_assets


    init_balance = 10000
    symbols = ['SPY','EFA','LQD','HYG', 'VNQ', 'REM','TLT','GLD', 'BIL']
    df = get_stock_data_with_ma(symbols=symbols, start_date='2016-08-01', end_date='2025-03-31', mas=[10], type='ma_month')
    df = filter_close_last_month(df)
    df = get_profit_ratio_avg(df, [12])

    for symbol, df_symbol in df.items():
        df[symbol] = df_symbol.dropna(subset=['Avg Profit Ratio'])

    df = merge_to_dfs(df)

    columns = [col for col in df.columns if col not in ['Adj Close','Close','Dividends','Condition', 'Interval']]
    df = df[columns]

    df['End Balance'] = None
    df['Balance'] = None
    df['Restart Asset'] = df.apply(get_selected_assets, axis=1) 
    df['Restart Balance'] = None

    for idx in range(len(df)):
        restart_asset = df.at[idx, 'Restart Asset']

        if idx == 0:
            balance = init_balance
            df.at[idx, 'Balance'] = balance
        else:
            prev_restart_balance = df.at[idx-1, 'Restart Balance']
            prev_restart_asset = df.at[idx-1, 'Restart Asset']
            
            end_balance = []
            for i in range(len(prev_restart_balance)):
                symbol_idx = df.at[0, 'Symbol'].index(prev_restart_asset[i])
                prev_close = df.at[idx-1,'Result Close'][symbol_idx]
                curr_close = df.at[idx, 'Result Close'][symbol_idx]
                change_rate = (curr_close - prev_close) / prev_close
                symbol_val = int(prev_restart_balance[i] *(1+ change_rate))
                end_balance.append(symbol_val)

            balance = sum([b for b in end_balance])

            df.at[idx, 'End Balance'] = end_balance
            df.at[idx, 'Balance'] = balance
        
        split_balance = int(balance / len(restart_asset))
        restart_balance = []
        if len(restart_asset) > 0:
            for _ in  range(len(restart_asset)):
                restart_balance.append(split_balance)
                balance -= split_balance

        df.at[idx, 'Restart Balance'] = restart_balance

    # display(df)
    df = get_performance(df)
    display(df)

composite_dual_momentum()

,Symbol,Start Date,End Date,Start Balance,End Balance,Annualized Return (CAGR),Standard Deviation,Sharpe Ratio,Maximum Drawdown
0,"[SPY, EFA, LQD, HYG, VNQ, REM, TLT, GLD, BIL]",2017-08-31,2025-03-28,10000,11324,0.016532,0.071027,-0.015132,-0.141246
